
# Lab: Detecção de Anomalias em Colunas de Plataformas Offshore (Tempo Quase Real, SGD Online)

Este laboratório cria um pipeline **reprodutível** para simular sinais de uma estrutura offshore,
extrair **features** em janelas deslizantes e detectar **anomalias** com modelos rasos atualizados
**online** via *Stochastic Gradient Descent (SGD)*.

**Componentes principais**:
- Simulador SDOF/2DOF com injeção de dano (queda de rigidez)
- Extração de frequência natural estimada (f₁) via Welch/peak-picking + features básicas
- Baseline físico: regressão linear multivariada (SGD) para predizer f₁ dado mar/temperatura/strain
- Autoencoder raso (SGD) para reconstrução de features; score = erro de reconstrução
- Fusão de scores + *threshold* adaptativo por quantil móvel
- Métricas: atraso de detecção, FPR, (opcional) ROC em cenários simulados
- Visualização com **matplotlib** (sem seaborn)

> Observação: GD (gradiente descendente) é **método de treino**, não a detecção em si.
> A detecção é feita comparando **observado vs. previsto/reconstruído**.


In [ ]:

# --- Setup & imports ---
import numpy as np
import pandas as pd
from scipy.signal import welch
import matplotlib.pyplot as plt
import math
import time

import torch
import torch.nn as nn
import torch.optim as optim

# Para reproducibilidade (pode variar intencionalmente para testar robustez)
np.random.seed(42)
torch.manual_seed(42)

print("Versões: numpy", np.__version__, "| pandas", pd.__version__, "| torch", torch.__version__)


## Simulador SDOF com injeção de dano

In [ ]:

def simulate_sdof(T=1800, fs=200, m=1.0, c=0.02, k=1000.0, damage_t=900, delta=0.02,
                   sea_noise_level=0.1, accel_noise_level=0.02):
    """
    Simula um SDOF excitado por ruído (mar) e injeta dano como queda de rigidez.
    Retorna: aceleração, strain (proxy), temperatura, vento, fs, vetor de rótulos 'is_damaged' (0/1).
    """
    n = T*fs
    dt = 1.0/fs
    x = np.zeros(n); v = np.zeros(n); a = np.zeros(n)
    k_t = k
    # Forcing pseudo-mar: ruído branco suavizado + componente baixa-freq
    forcing = sea_noise_level*np.random.randn(n) + 0.02*np.sin(np.linspace(0, 40*math.pi, n))
    strain = np.zeros(n)
    is_damaged = np.zeros(n, dtype=int)

    for i in range(1,n):
        if i == int(damage_t*fs):
            k_t = k*(1.0-delta)
        if i >= int(damage_t*fs):
            is_damaged[i] = 1
        a[i] = (forcing[i] - c*v[i-1] - k_t*x[i-1]) / m
        v[i] = v[i-1] + a[i]*dt
        x[i] = x[i-1] + v[i]*dt
        # Strain proxy proporcional a deslocamento * rigidez instantânea (simplificação)
        strain[i] = x[i] * k_t

    accel = a + accel_noise_level*np.random.randn(n)
    # Temperatura e vento como covariáveis lentas
    temp = 20 + 5*np.sin(np.linspace(0, 6*math.pi, n)) + 0.2*np.random.randn(n)
    wind = 5 + 2*np.sin(np.linspace(0, 10*math.pi, n)) + 0.5*np.random.randn(n)
    return accel, strain, temp, wind, fs, is_damaged


## Extração de features por janelas (f₁ via Welch, RMS, tendências)

In [ ]:

def estimate_f1(accel_win, fs, fmin=0.1, fmax=50.0):
    f, Pxx = welch(accel_win, fs=fs, nperseg=min(len(accel_win)//2, 512))
    mask = (f >= fmin) & (f <= fmax)
    if not np.any(mask):
        return np.nan
    f_slice = f[mask]
    P_slice = Pxx[mask]
    idx = np.argmax(P_slice)
    return float(f_slice[idx])

def extract_features(accel_win, strain_win, temp_win, wind_win, fs):
    f1 = estimate_f1(accel_win, fs)
    rms = float(np.sqrt(np.mean(accel_win**2)))
    # Tendências lineares simples
    x_idx = np.arange(len(strain_win), dtype=float)
    st_mean = float(np.mean(strain_win))
    st_slope = float(np.polyfit(x_idx, strain_win, 1)[0]) if len(strain_win) > 1 else 0.0
    t_mean = float(np.mean(temp_win))
    w_mean = float(np.mean(wind_win))
    return np.array([f1, rms, st_mean, st_slope, t_mean, w_mean], dtype=np.float32)


## Modelos rasos com treino online (SGD)

In [ ]:

class LinearBaseline(nn.Module):
    """Prediz f1 a partir de [rms, st_mean, st_slope, t_mean, w_mean, bias]."""
    def __init__(self, d):
        super().__init__()
        # Peso único: y_hat = w^T u  (implementado como nn.Linear sem bias explícito)
        self.lin = nn.Linear(d, 1, bias=False)
        # Inicialização leve
        nn.init.zeros_(self.lin.weight)

    def forward(self, u):
        return self.lin(u).squeeze(-1)

class AE(nn.Module):
    def __init__(self, d_in=5, h1=16, z=8):
        super().__init__()
        self.enc = nn.Sequential(nn.Linear(d_in, h1), nn.ReLU(), nn.Linear(h1, z))
        self.dec = nn.Sequential(nn.Linear(z, h1), nn.ReLU(), nn.Linear(h1, d_in))

    def forward(self, x):
        z = self.enc(x)
        xrec = self.dec(z)
        return xrec


## Loop online: janelas deslizantes, atualização SGD, scores e threshold

In [ ]:

# Parâmetros de simulação e janelamento
T = 1800          # segundos totais
fs = 200          # Hz
win_sec = 4.0
overlap = 0.5
win = int(win_sec*fs)
step = int(win*(1.0-overlap))

accel, strain, temp, wind, fs, is_damaged = simulate_sdof(T=T, fs=fs, damage_t=900, delta=0.02)

# Modelos & otimizadores
# Baseline linear: entrada = [rms, st_mean, st_slope, t_mean, w_mean, bias]
d_u = 6  # 5 features + 1 bias
baseline = LinearBaseline(d=d_u)
opt_lin = optim.SGD(baseline.parameters(), lr=1e-3)

# Autoencoder: entrada = [rms, st_mean, st_slope, t_mean, w_mean]
ae = AE(d_in=5, h1=16, z=8)
opt_ae = optim.SGD(ae.parameters(), lr=1e-3)

hist = []  # iremos guardar dicts com métricas por janela
scores_hist = []

for start in range(0, len(accel)-win, step):
    sl = slice(start, start+win)
    accw, stw, tw, ww = accel[sl], strain[sl], temp[sl], wind[sl]
    feats = extract_features(accw, stw, tw, ww, fs)
    f1 = feats[0]
    # Se f1 falhar (nan), pule atualização para esta janela
    if not np.isfinite(f1):
        continue

    # Constrói u para baseline (com bias)
    feats_lin = feats[1:]  # [rms, st_mean, st_slope, t_mean, w_mean]
    u = np.r_[feats_lin, 1.0].astype(np.float32)
    u_t = torch.from_numpy(u).unsqueeze(0)  # (1,6)
    y_t = torch.tensor([f1], dtype=torch.float32)  # (1,)

    # --- Baseline linear: y_hat ~ f1
    opt_lin.zero_grad()
    y_hat = baseline(u_t)  # (1,)
    loss_lin = (y_hat - y_t).pow(2).mean()
    loss_lin.backward()
    opt_lin.step()

    # --- Autoencoder: reconstruir x_in = feats_lin
    x_in = torch.from_numpy(feats_lin).unsqueeze(0)  # (1,5)
    opt_ae.zero_grad()
    x_rec = ae(x_in)
    loss_ae = ((x_rec - x_in)**2).mean()
    loss_ae.backward()
    opt_ae.step()

    # --- Score de anomalia (combinação simples)
    with torch.no_grad():
        y_hat_detached = baseline(u_t).item()
        score_pred = abs(f1 - y_hat_detached)
        score_rec = float(loss_ae.item())
        score = score_pred + score_rec
        scores_hist.append(score)

        # threshold adaptativo por quantil móvel (99º), usando ~ 10 min de histórico por padrão
        # 10 min de dados => janelas por minuto ~ (60/((win_sec*(1-overlap)))) ; arredondamos para ~120
        thr = np.inf
        if len(scores_hist) > 120:
            thr = float(np.quantile(scores_hist[-120:], 0.99))
        alert = (score > thr)

    # Marca temporal da janela (segundos)
    t_mid = (start + win//2) / fs
    damaged = int(np.any(is_damaged[sl]))

    hist.append(dict(
        t=t_mid, f1_obs=float(f1), f1_pred=float(y_hat_detached), 
        loss_ae=float(loss_ae.item()), score=float(score), thr=float(thr), 
        alert=bool(alert), damaged=damaged
    ))

df = pd.DataFrame(hist)
print("Amostras de janelas:", len(df))
df.head()


## Visualizações

In [ ]:

# Plot 1: f1 observado vs previsto
plt.figure()
plt.plot(df['t'], df['f1_obs'], label='f1_obs')
plt.plot(df['t'], df['f1_pred'], label='f1_pred')
plt.xlabel('Tempo (s)')
plt.ylabel('f1 (Hz)')
plt.title('Frequência natural observada vs prevista (baseline linear)')
plt.legend()
plt.show()


In [ ]:

# Plot 2: Score vs Threshold + janela de dano
plt.figure()
plt.plot(df['t'], df['score'], label='score')
plt.plot(df['t'], df['thr'], label='threshold')
# Sombrear região de dano
if df['damaged'].any():
    t_damage_start = df.loc[df['damaged']==1, 't'].min()
    plt.axvspan(t_damage_start, df['t'].max(), alpha=0.2)
plt.xlabel('Tempo (s)')
plt.ylabel('Score')
plt.title('Score de anomalia e limiar adaptativo')
plt.legend()
plt.show()


In [ ]:

# Plot 3: Série de alertas (marcadores)
plt.figure()
plt.plot(df['t'], df['alert'].astype(int), drawstyle='steps-post')
plt.xlabel('Tempo (s)')
plt.ylabel('Alerta (0/1)')
plt.title('Alertas gerados')
plt.show()


## Métricas: atraso de detecção e FPR

In [ ]:

# Atraso até 1º alerta após início do dano
det_delay = np.nan
if df['damaged'].any() and df['alert'].any():
    t_damage = df.loc[df['damaged']==1, 't'].min()
    after = df[df['t'] >= t_damage]
    fired = after[after['alert'] == True]
    if len(fired) > 0:
        det_delay = float(fired['t'].iloc[0] - t_damage)

# FPR: alertas quando 'damaged'==0 dividido por tempo pré-dano
pre = df[df['damaged']==0]
fpr_per_hour = np.nan
if len(pre) > 0:
    time_pre = pre['t'].max() - pre['t'].min()
    alerts_pre = int(pre['alert'].sum())
    if time_pre > 0:
        fpr_per_hour = float(alerts_pre / (time_pre/3600.0))

print("Atraso de detecção (s):", det_delay)
print("Falsos positivos por hora (aprox):", fpr_per_hour)



## Extensões sugeridas
- **Peak tracking/Kalman** para f₁ reduzir ruído de estimativa
- **Quantil condicional**: thresholds por estado de mar/vento
- **2DOF/3DOF** no simulador para multi-modos acoplados
- **MQTT/Kafka** para streaming real; consumidor executa este loop online
- **Banco temporal (InfluxDB/Timescale)** + **Grafana** para dashboard operacional

## Suposições e validações
- Queda de f₁ pode vir de **temperatura/preload**; o baseline multivariado ajuda a filtrar
- AE raso pode **aprender drift**: use janela de treino curta e compare sempre com baseline físico
- FFT em janelas curtas: use **Welch/médias**; se necessário, suavize com filtro/trackers
